In [48]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [49]:
X_cents = np.load('dataset/cents.npy')
X_counts = np.load('dataset/counts.npy')
y = np.load('dataset/labels.npy')

In [50]:
print(f'X_cents shape: {X_cents.shape}')
print(f'X_counts shape: {X_counts.shape}')
print(f'y shape: {y.shape}')

X_cents shape: (149084, 1024)
X_counts shape: (149084, 73)
y shape: (149084, 40)


In [51]:
X_cents_train, X_cents_temp, X_counts_train, X_counts_temp, y_train, y_temp = train_test_split(X_cents, X_counts, y, test_size=0.3, random_state=42)
X_cents_val, X_cents_test, X_counts_val, X_counts_test, y_val, y_test = train_test_split(X_cents_temp, X_counts_temp, y_temp, test_size=0.5, random_state=42)

## DeepGRU

In [52]:
seq_input = keras.layers.Input(shape=(None, 1), name='Sequence input')
static_input = keras.layers.Input(shape=(X_counts.shape[1],), name='Static input')

In [53]:
# Encoder Network
#x = keras.layers.GRU(128, activation='relu', return_sequences=True)(seq_input)
# = keras.layers.GRU(64, activation='relu')(x)

# Attention module
#x = keras.layers.Attention()([x, x])

In [54]:
#x = keras.layers.Dense(128, activation='relu')(x)
#x = keras.layers.Concatenate()([x, static_input])
#x = keras.layers.Dense(64, activation='relu')(x)
#output = keras.layers.Dense(y.shape[1], activation='softmax', name='output')(x)

In [58]:
x = keras.layers.Dense(512, activation='relu')(static_input)
x = keras.layers.Dense(512, activation='relu')(x)
x = keras.layers.Dense(256, activation='relu')(x)
x = keras.layers.Dense(256, activation='relu')(x)
x = keras.layers.Dense(128, activation='relu')(x)
x = keras.layers.Dense(64, activation='relu')(x)
output = keras.layers.Dense(y.shape[1], activation='softmax', name='output')(x)

deepGRU = keras.models.Model(static_input, outputs=output, name='DeepGRU')

In [59]:
deepGRU.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
deepGRU.summary()

Model: "DeepGRU"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Static input (InputLayer)       │ (None, 73)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 512)            │        37,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 40)             │         2,600 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 541,416 (2.07 MB)

 Trainable params: 541,416 (2.07 MB)

 Non-trainable params: 0 (0.00 B)

In [62]:
deepGRU.fit(X_counts_train, y_train, validation_data=(X_counts_val, y_val), epochs=50)

Epoch 1/50
3262/3262 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.7302 - loss: 0.9190 - val_accuracy: 0.4264 - val_loss: 2.9673
Epoch 2/50
3262/3262 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.7383 - loss: 0.8953 - val_accuracy: 0.4330 - val_loss: 3.0975
Epoch 3/50
3262/3262 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.7429 - loss: 0.8778 - val_accuracy: 0.4305 - val_loss: 2.9754
Epoch 4/50
3262/3262 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.7421 - loss: 0.8795 - val_accuracy: 0.4229 - val_loss: 2.9790
Epoch 5/50
3262/3262 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.7361 - loss: 0.9037 - val_accuracy: 0.4336 - val_loss: 3.0201
Epoch 6/50
3262/3262 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.7427 - loss: 0.8769 - val_accuracy: 0.4321 - val_loss: 3.0795
Epoch 7/50
3262/3262 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.7466 - loss: 0.8644 - val_accuracy: 0.4270 - val_loss: 3.1243
Epoch 8/50
3262/3262 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.7441 - loss: 0.8766 

In [ ]:
deepGRU.save('models/deepGRU.h5')